## **Lernziele**

Durch das Bearbeiten dieser Übungen werden Sie:

- Retrieval-Augmented Generation (RAG) und seine Komponenten verstehen.
- PDF-Dokumente effektiv laden, vorverarbeiten und verarbeiten.
- Textdaten in Embeddings für effiziente Suche umwandeln.
- Dokumenten-Retrieval-Systeme mit LangChain und FAISS implementieren und testen.
- Retrieval-Systeme mit kostenlosen Sprachmodellen (LLMs) von ChatGroq integrieren.
- Ein interaktives chatbasiertes Frage-Antwort-System aufbauen.

---

## **Übung 1: Setup und Aufwärmen**

In dieser Übung richten Sie Ihre Umgebung ein und wählen ein geeignetes Sprachmodell aus.

**Schritte:**

1. **Umgebungsvariablen laden:** Stellen Sie sicher, dass Ihre Umgebungsvariablen (z.B. API-Schlüssel, Tokens) sicher gespeichert und geladen werden.
2. **LLM auswählen:** Wählen Sie ein kostenloses LLM-Modell von ChatGroq aus.
3. **Modell instanziieren:** Erstellen Sie eine Instanz Ihres gewählten Modells.

In [1]:
# Import necessary libraries
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEndpoint

# Load environment variables
load_dotenv()

c:\Users\stf\workspace_neuefische\Chatbot-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# warnings.filterwarnings("ignore")
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant", #"llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

---

## **Übung 2: Datenaufnahme**

In dieser Übung lernen Sie, PDF-Daten in eine Python-Umgebung zu laden.

**Schritte:**

1. **PDF-Loader importieren:** Verwenden Sie LangChains `PyPDFLoader`.
2. **PDF-Datei laden:** Erstellen Sie eine Funktion zum Lesen der PDF-Datei.
3. **PDF-Inhalt anzeigen:** Geben Sie die Seitenanzahl und den Inhalt der ersten Seite aus.

In [3]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from pathlib import Path

data_path = Path("data") 

loader = DirectoryLoader(
    str(data_path),
    glob="**/*.md",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
)

docs = loader.load()

print(f"{len(docs)} Dokumente geladen")
    

23 Dokumente geladen


In [4]:
print(docs[3].page_content)

# Tag 07, 27.10.2023

Logistische Regression

---

## Grundlegender Überblick

- **Überwachtes Lernen**: Nutzt gelabelte Daten zur Modellbildung.
- **Klassifikation vs. Regression**:
  - Klassifikation ordnet Eingaben diskreten Klassen zu (z.B. Spam/kein Spam).
  - Regression sagt kontinuierliche Werte voraus (z.B. Temperatur, Preis).
- **Klassifikationsarten**:
  - Binär: Zwei Klassen (z.B. traurig/glücklich).
  - Multiklassig: Mehr als zwei Klassen (z.B. Katze/Hund/Krokodil).
  - Multilabel: Ein Input kann mehreren Klassen zugeordnet werden (z.B. Filmgenres).
- **Evaluierungsmetriken**:
  - Konfusionsmatrix: True/False Positives/Negatives.
  - Accuracy: Anteil korrekter Vorhersagen (irreführend bei unausgewogenen Daten).
  - Precision: Anteil korrekter positiver Vorhersagen.
  - Recall: Anteil erkannter positiver Fälle.
  - F1-Score: Harmonisches Mittel aus Precision und Recall.
- **Wahrscheinlichkeiten & Threshold**:
  - Klassifikationsmodelle liefern oft Wahrscheinlichkeiten (z.B. 

---

## **Übung 3: Dokument-Chunking**

Diese Übung führt das Aufteilen großer Dokumente in handhabbare Textblöcke ein.

**Schritte:**

1. **Text-Splitter importieren:** Verwenden Sie `RecursiveCharacterTextSplitter`.
2. **Dokument aufteilen:** Schreiben Sie eine Funktion, die geladene Dokumente in Chunks aufteilt.
3. **Funktion testen:** Überprüfen Sie das Ergebnis, indem Sie die resultierenden Chunks anzeigen.

In [5]:
react_docs = docs

In [66]:
import re

In [69]:
# Import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_documents(documents, chunk_size=400, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents=documents)
    
    # Just to add id for etch chunks to map it later 
    for i, chunk in enumerate(chunks):
        days = re.findall(r"Tag \d{1,2}", chunk.page_content)
        chunk.metadata.update({
            "id": f"chunk_{i}"
            })
        if len(days) > 0:
            chunk.metadata.update({
                "day": days[0]
            })
            print(days[0])
    
    return chunks


react_chunks = chunk_documents(react_docs)

Tag 03
Tag 04
Tag 06
Tag 07
Tag 08
Tag 9
Tag 11
Tag 13
Tag 14
Tag 16
Tag 16
Tag 17
Tag 18
Tag 21
Tag 22
Tag 23
Tag 24
Tag 25
Tag 27
Tag 28
Tag 10
Tag 12
Tag 26


In [51]:
react_chunks[0]

Document(metadata={'source': 'data\\protocol_26-01-21.md', 'id': 'chunk_0'}, page_content='# Tag 03, 21.01.2026\n\nExplorative Datenanalyse (kurz EDA)\n\n---\n## __Grundlegender Überblick__\n\n* Grundlegende Begrifflichkeiten in der EDA\n* Was ist EDA und warum machen wir das?\n* Arten von EDA\n* Schätzungen\n* Datentypen\n* Univariate EDA - Berechnungen von Mittelwert, Median und Modus\n* Streuung von Daten\n\n---\n##  __Zeitplan__')

In [52]:
# Execute your chunking function and display results here
len(react_chunks)

631


---

## **Übung 4: Embedding und Speicherung**

In dieser Übung erstellen Sie Embeddings aus Textblöcken und speichern sie effizient.

**Schritte:**

1. **Embedding-Modell wählen:** Verwenden Sie `sentence-transformers/all-mpnet-base-v2` von Hugging Face.
2. **Embeddings generieren:** Wandeln Sie Dokument-Chunks in Embeddings um.
3. **Embeddings speichern:** Speichern Sie diese Embeddings lokal mit FAISS.

In [53]:
# Import libraries
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

def embed_and_store(chunks, db_name):
    """
    This function uses the open-source embedding model HuggingFaceEmbeddings 
    to create embeddings and store those in a VectorStore called FAISS, 
    which allows for efficient similarity search
    """
    # instantiate embedding model
    embedding = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2',
        encode_kwargs={"normalize_embeddings": True}
    )
    # create the vector store 
    vectorstore = FAISS.from_documents(
        documents=chunks,
        embedding=embedding,
        distance_strategy=DistanceStrategy.COSINE  # or DistanceStrategy.DOT or DistanceStrategy.L2 
        
    )
    # save VectorStore locally
    vectorstore.save_local(f"../vector_databases/vector_db_{db_name}")
    return vectorstore



In [54]:
# Generate embeddings and save them locally
all_embedding=embed_and_store(chunks=react_chunks, db_name="chatbot")

---

## **Übung 5: Retrieval aus FAISS**

Hier lernen Sie, wie man Dokumente aus einer Vektordatenbank mithilfe von Embeddings abruft.

**Schritte:**

1. **Embeddings laden:** Laden Sie gespeicherte Embeddings aus der FAISS-Datenbank.
2. **Retrieval implementieren:** Erstellen Sie eine Logik zum Abrufen relevanter Chunks basierend auf Abfragen.
3. **Retriever testen:** Führen Sie die Suche mit Beispielabfragen durch.

In [55]:
# Implement retrieval logic from your FAISS database
def retrieve_from_vector_db(vector_db_path):
    """
    this function splits out a retriever object from a local VectorStore
    """
    # instantiate embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2',
        encode_kwargs={"normalize_embeddings": True}
    )
    react_vectorstore = FAISS.load_local(
        folder_path=vector_db_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True,
        distance_strategy=DistanceStrategy.COSINE  # or DistanceStrategy.DOT or DistanceStrategy.L2 
    )
    retriever = react_vectorstore.as_retriever()
    return retriever ,react_vectorstore

# Load the retriever and index
react_retriever,react_vectorstore = retrieve_from_vector_db("../vector_databases/vector_db_chatbot")
type(react_retriever),type(react_vectorstore)

(langchain_core.vectorstores.base.VectorStoreRetriever,
 langchain_community.vectorstores.faiss.FAISS)

In [56]:
# Test your retrieval system with queries
react_retriever.get_relevant_documents("Anekdote",k=3)

[Document(id='cfecc979-f5c4-4ea5-91da-7145de1fe01c', metadata={'source': 'data\\protocol_26-02-18_clustering.md', 'id': 'chunk_429'}, page_content='---\n\n## Anekdote\n\nAutor Borges  \nEnzyklopädie "Emporio celestial de conocimientos benévolos"  \nKlassifizierung von  Tieren, wenn auch nach eher absurden Kriterien\n\n---\n\n## Haupt Take-Away'),
 Document(id='91f2064e-50de-4c10-bfab-30dc96c284aa', metadata={'source': 'data\\protocol_26_02_23.md', 'id': 'chunk_622'}, page_content='---\n##  __Zeitplan__\n\n|Zeit|Inhalt|\n|---|---|\n|09:00 - 09:25|Daily Review|\n|10:00 - 11:15|Erste theoretische Inputs|\n|11:15 - 12:15|Mittagspause| \n|13:30 - 16:00|Praktische Übungen|\n|16:00 - 16:30|Tägliches Stand-up|\n|16:30 - 18:00|Übungen und Abschluss|\n\n\n---\n## __Erste inhaltliche Notizen__\n\n\n### Anekdote\nDie Fluglinie Air Canada, deren ChatBot behauptete 90 Tage dauerte  eine Rücküberweisung'),
 Document(id='67d8d7cb-deb2-4f6e-9d4d-57ea83544912', metadata={'source': 'data\\protocol_26-02-

---

## **Übung 6: Retrieval mit LLM verbinden**

Nun verbinden Sie das Dokument-Retrieval mit dem Sprachmodell.

**Schritte:**

1. **Retrieval-Chain erstellen:** Verknüpfen Sie Ihr Retrieval-System mit Ihrem instanziierten LLM.
2. **Chain testen:** Bestätigen Sie die Funktionalität, indem Sie Antworten aus abgerufenen Dokumenten generieren.

In [57]:
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

# Write a function to create retrieval and document processing chains
def connect_chains(retriever):
    """
    this function connects stuff_documents_chain with retrieval_chain
    """
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat")
    )
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain


react_retrieval_chain = connect_chains(react_retriever)

In [65]:
# Invoke your chain with a sample question
output = react_retrieval_chain.invoke(
    {"input": "In den Metadaten ist day angegeben und mit Werten wie z.B. 'Tag 9' gefüllt. Welches Datum war das Thema von Tag 21?'"}
)
answer_list = output['answer'].split(".")
for sentence in answer_list:
    print(sentence)

Das Datum des Themas von Tag 21 ist nicht explizit angegeben
 Es gibt jedoch eine Zeile "Tag 21, 16
02
2026" am Anfang des Kontexts, die wahrscheinlich das Datum des Themas von Tag 21 ist



---

## **Übung 7: Interaktives Chat-System**

In der letzten Übung bauen Sie ein interaktives chatbasiertes Abfragesystem.

**Schritte:**

1. **Chat-Oberfläche erstellen:** Entwickeln Sie eine einfache Funktion für interaktive Abfragen.
2. **Chat ausführen:** Ermöglichen Sie es Nutzern, Fragen zu stellen und sofortige Antworten zu erhalten.

In [ ]:
# Define your interactive chat querying function

while True:
    # 1. Do something (runs at least once)
    user_query = input("Frage ans RAG-System: ")
    
    # 2. Check the "until" condition
    if user_query == 'quit':
        break
    print(react_retrieval_chain.invoke({"input":user_query})['answer'])
    print("...")

Ja, das weltweite Wirtschaftswachstum hat sich insgesamt weiterhin als robust erwiesen. Dies ist zu verdanken an:

1. Kräftiges Wirtschaftswachstum im dritten Quartal 2025
2. Robuste Nachfrage nach höher rentierenden Staatsanleihen

Diese Faktoren haben dazu beigetragen, dass das weltweite Wirtschaftswachstum stabil und kräftig bleibt.
...
Das Dokument sagt, dass der Einkaufsmanagerindex (EMI) für die Produktion im verarbeitenden Gewerbe und im Dienstleistungssektor im Januar 2026 weitgehend stabil geblieben ist.
...
Es scheint, dass der bereitgestellte Text nicht explizit fünf Kernaussagen enthält. Der Text scheint eher ein Fragment zu sein, das verschiedene Ideen und Aussagen enthält, aber keine klaren Kernaussagen. Es gibt jedoch einige Schlüsselaussagen, die ich herausgreifen kann:

1. Die Fähigkeiten, Kenntnisse und Kompetenzen für Aktivitäten mit Brennstoffen sind nicht ein separates Spektrum an Fähigkeiten.
2. Brennstoffe mit höheren Grenzkosten führen tendenziell zu höheren Str

In [26]:
# Run and test your interactive chat system

---

## **Fazit & Reflexion**

Nach Abschluss dieser Übungen:

- Fassen Sie die gelernten Schlüsselkonzepte zusammen.
- Reflektieren Sie über die Wirksamkeit und Einschränkungen des kostenlosen LLM und des von Ihnen aufgebauten RAG-Systems.
- Überlegen Sie, wie Sie Ihr System in praktischen Anwendungen verbessern oder erweitern könnten.

---